### Test of data instantiation

In [1]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import pandas as pd
from PIL import Image
import os


In [2]:

        
# img = torch.ones(6, 3)
# labels = [0, 1, 1, 1, 0, 0]

# dataset = ExpDataSet(img, labels)
# print(dataset[4])



In [15]:
# Local paths

label_path = "dataset/datasets/ringmerkingno/ringcodes.csv"

# Make sure to separate csv file on |
df = pd.read_csv(label_path, sep="|")

df.head(10)

,filename,code,color
0,LBNW(J22M)_20080310234907_79.jpg,J22M,NW
1,LBNW(J03Y)_20080311094828_79.jpg,J03Y,NW
2,LBBW(JR0T)_20080320193604_79.jpg,JR0T,BW
3,LBBW(J1EM)_20080321135128_79.jpg,J1EM,BW
4,LBBW(JZ3K)_20111027125715_79.jpg,JZ3K,BW
5,LBBW(JT0G)_20141104000055_79.jpg,JT0G,BW
6,LBPN(J512)_20090309004214_79.jpg,J512,PN
7,LBBW(J9EE)_20101214162613_79.jpg,J9EE,BW
8,LBBW(J9EE)_20101214162645_79.jpg,J9EE,BW
9,LBBW(J00X)_20090309175308_79.jpg,J00X,BW


In [ ]:

dl = df.sort_values("filename", ascending=True).reset_index(drop=True)
dl.head(10)
# exp_labels = dl[["code", "color"]]
# exp_labels.head()


,code,color
0,WT,BW
1,J0K8,WN
2,J0K8,WN
3,JA0H,WN
4,CN,NW


In [5]:
class ExpDataSet(Dataset):
    def __init__(self, img_path:str, labels_path:str, transform=None, max_n=None):
        self.img_path = img_path
        self.transform = transform
        self.max_n = max_n

        # Retrieve labels and sort them in alphabetical order to match the images
        df = pd.read_csv(labels_path, sep="|")
        df = df.sort_values("filename", ascending=True).reset_index(drop=True)

        # Ensure the data contains the chosen amount of elements
        if max_n is not None:
            df = df[:self.max_n]
        
        # Retrieve images from folder and match them with labels
        self.img_paths = df["filename"].apply(lambda e: os.path.join(img_path, e)).to_list()

        self.labels = list(df[["code","color"]].itertuples(index=False, name=None)) # Labels should be a list of tuples
        
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = Image.open(self.img_paths[idx]).convert('RGB')

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label

In [11]:
image_path = "dataset/datasets/ringmerkingno/images"
max_n = 10
exp_dataset = ExpDataSet(img_path=image_path, labels_path=label_path, max_n=max_n)




In [14]:
for i in range(0, max_n):
    print(exp_dataset[i])

(<PIL.Image.Image image mode=RGB size=1200x948 at 0x2630025A2A0>, ('WT', 'BW'))
(<PIL.Image.Image image mode=RGB size=1024x768 at 0x26342D62D50>, ('J0K8', 'WN'))
(<PIL.Image.Image image mode=RGB size=1024x768 at 0x26342D62D50>, ('J0K8', 'WN'))
(<PIL.Image.Image image mode=RGB size=1024x768 at 0x26342D62D50>, ('JA0H', 'WN'))
(<PIL.Image.Image image mode=RGB size=1200x869 at 0x26342D62D50>, ('CN', 'NW'))
(<PIL.Image.Image image mode=RGB size=1200x855 at 0x26342D62D50>, ('232', 'BO'))
(<PIL.Image.Image image mode=RGB size=1200x891 at 0x26342D62D50>, ('924', 'BO'))
(<PIL.Image.Image image mode=RGB size=1200x783 at 0x26342D62D50>, ('CZP', 'BO'))
(<PIL.Image.Image image mode=RGB size=1200x1039 at 0x26342D62D50>, ('CZP', 'BO'))
(<PIL.Image.Image image mode=RGB size=949x581 at 0x26342D62D50>, ('210E', 'BW'))
